In [1]:
import numpy as np
from math import sqrt
import gzip
import shutil
import georinex as gr
#from scipy.linalg import inv
import pandas as pd

In [13]:
rinex = pd.read_csv('Data/rnx_010000.csv')
sp3 = pd.read_csv('Data/sp3_010000.csv')
clock = pd.read_csv('Data/clk_010000.csv')

rinex_clean = rinex[rinex['C1'].notna()].copy()
rinex_clean['sat'] = rinex_clean['Sat']

merge1 = pd.merge(rinex_clean, sp3, on=['sat'], how='left')
merged = pd.merge(merge1, clock, on=['sat'], how='left')

c = 299792458.0

merged.columns

Index(['Unnamed: 0', 'Time_x', 'Sat', 'C1', 'C2', 'L1', 'L2', 'P2', 'S1', 'S2',
       'sat', 'x', 'y', 'z', 'clk', 'dist', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'code', 'Time_y', 'dunno', 'clk_offset',
       'clk_rate'],
      dtype='object')

In [38]:
import numpy as np
import pandas as pd

c = 299792458
initial_guess = np.array([4750000.0, -5050000.0, -350000.0, 0.0])


df = merged
pseudoranges = df['C1'].values
sat_positions = df[['x', 'y', 'z']].values *1000  # Already in meters, no * 1000
sat_clock_biases = df['clk_offset'].values


def compute_geometry_matrix(rec_pos, sat_pos):
    G = np.zeros((len(sat_pos), 4))
    for i, pos in enumerate(sat_pos):
        dx = pos[0] - rec_pos[0]
        dy = pos[1] - rec_pos[1]
        dz = pos[2] - rec_pos[2]
        r = np.sqrt(dx**2 + dy**2 + dz**2)
        if r < 1e-10: r = 1e-10
        G[i, 0] = -dx / r
        G[i, 1] = -dy / r
        G[i, 2] = -dz / r
        G[i, 3] = c
    return G

def compute_predicted_pseudoranges(rec_pos, sat_pos, sat_clocks, rec_clock):
    predicted = np.zeros(len(sat_pos))
    for i, pos in enumerate(sat_pos):
        r = np.sqrt((pos[0] - rec_pos[0])**2 + (pos[1] - rec_pos[1])**2 + (pos[2] - rec_pos[2])**2)
        predicted[i] = r + c * (rec_clock - sat_clocks[i])
    return predicted

def position_least_squares(pseudoranges, sat_positions, sat_clock_biases, initial_guess, max_iter=10, tol=0.1):
    x = initial_guess.copy()
    for iteration in range(max_iter):
        predicted = compute_predicted_pseudoranges(x[:3], sat_positions, sat_clock_biases, x[3] / c)
        residuals = pseudoranges - predicted
        G = compute_geometry_matrix(x[:3], sat_positions)
        rank = np.linalg.matrix_rank(G)
        if rank < 4:
            print(f"Iteration {iteration + 1}: Rank = {rank}")
            return None, None
        delta_x, _, _, _ = np.linalg.lstsq(G, residuals, rcond=None)
        x += delta_x
        if np.linalg.norm(delta_x[:3]) < tol:
            print(f"Converged after {iteration + 1} iterations")
            break
    return x[:3], x[3] / c

estimated_pos, clock_bias = position_least_squares(
    pseudoranges, sat_positions, sat_clock_biases, initial_guess
)

if estimated_pos is not None:
    print()
    print(f"Initial Rough Guess C211 Position at 2022-01-20 01:00:00 (ECEF):")
    print(f"x: {initial_guess[0]:.2f} m")
    print(f"y: {initial_guess[1]:.2f} m")
    print(f"z: {initial_guess[2]:.2f} m")
    
    print()

    print(f"Estimated C211 Position at 2022-01-20 01:00:00 (ECEF):")
    print(f"x: {estimated_pos[0]:.2f} m")
    print(f"y: {estimated_pos[1]:.2f} m")
    print(f"z: {estimated_pos[2]:.2f} m")
    print()
    print(f"Receiver Clock Bias: {clock_bias*1e6:.8f} microseconds")
    r = np.sqrt(np.sum(estimated_pos**2))
    print(f"Distance from Earth center: {r:.2f} m (Altitude: {(r - 6378000)/1000:.2f} km)")
else:
    print("Failed to compute position")

Converged after 3 iterations

Initial Rough Guess C211 Position at 2022-01-20 01:00:00 (ECEF):
x: 4750000.00 m
y: -5050000.00 m
z: -350000.00 m

Estimated C211 Position at 2022-01-20 01:00:00 (ECEF):
x: 4735425.49 m
y: -5038408.64 m
z: -329611.42 m

Receiver Clock Bias: -0.00000030 microseconds
Distance from Earth center: 6922316.08 m (Altitude: 544.32 km)
